# Seleccionar imatges sense nuvols

En aquest jupyter tractarem com descartar imatges amb nuvols d'un set d'imatges que poden o no tenir nuvols.

Ens podem trobar en necessitat d'aquest jupiter si fem servir per exemple el Sentinel3_OLCI_L1B, el qual no pot ser indexat per nubulositat, i volem tractar amb dades en les quals els nuvols estorben.

Per a fer aixó el procediment que seguim es molt simple:

1. Seleccionem un area, intèrval de temps, satèlit, bandes i nubulositat màxima que volem descarregar.
2. Descarreguem totes les dades descrites, sense importar els nuvols de moment.
3. Descarreguem totes les imatges del Sentinel2_L2A per a les mateixes coordenades i espai de temps, peró aquest cop filtrem per nubulositat, ja que el Sentinel2_L2A ens permet fer aixó.
4. Separem el subconjunt d'imatges dels 2 satèlits que comparteixen data.

Per a fer aixó hem donat per suposat que si en una imatge del Sentinel2 no hi han nuvols, en una del Sentinel3 tampoc, aixó evidentment és especulatiu i cadascú aplicarà el criteri que cregui.

#### ↓ Connexió i dades d'entrada ↓

En aquest apartat hem concentrat totes les variables que poden ser d'interès al lector/programador.

In [30]:
# --- Connexió
import openeo # https://openeo.org/documentation/1.0/python/#collections

# Connexió al "Backend" d'on extreurèm les dades
backend = "openeo.dataspace.copernicus.eu"
conn = openeo.connect(backend).authenticate_oidc()


# --- Filtratge de dades
satelit="SENTINEL3_OLCI_L1B"
spatial_extent = {
    "west": 0.266516,
    "east": 2.360285,
    "south": 40.120817,
    "north": 41.479864,
}
temporal_extent = ["2023-01-01", "2024-01-01"] # Les dates estàn en notació: aaaa-mm-dd 
bandes_finals = ["B03", "B04", "B05", "B09", "B10"]

max_cloud_coverage = 0 # Variar aquest numero per retocar la tolerància de les imatges


# --- Directoris i fitxers
directori_temp_dades_finals = "output/dades_finals_sense_filtrar/"
directori_temp_sentinel2_sense_nuvols = "output/sentinel2_sense_nuvols/"
directori_dades_finals = "output/dades_finals_filtrades/"

Authenticated using refresh token.


#### ↓ Dades que necessitem ↓

In [31]:
datacube = conn.load_collection(
    satelit,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bandes_finals, # Bandas para detectar clorofila y algas
)
 
datacube = datacube.apply(lambda x: x / 10000) # Normalitzem les dades (se suposa que estan 0-10.000)

result = datacube.save_result("GTiff") 

job = result.create_job()
job.start_and_wait()
job.get_results().download_files(directori_temp_dades_finals)

0:00:00 Job 'j-240914600fdc42a491f90b5c3c832fbb': send 'start'
0:00:13 Job 'j-240914600fdc42a491f90b5c3c832fbb': created (progress 0%)
0:00:18 Job 'j-240914600fdc42a491f90b5c3c832fbb': created (progress 0%)
0:00:25 Job 'j-240914600fdc42a491f90b5c3c832fbb': created (progress 0%)
0:00:33 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:00:43 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:00:55 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:01:11 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:01:30 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:01:54 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:02:24 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:03:05 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:03:52 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progress N/A)
0:04:50 Job 'j-240914600fdc42a491f90b5c3c832fbb': running (progr

[PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-01Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-02Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-03Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-04Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-05Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-06Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-07Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-08Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-09Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-10Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-11Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-12Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-13Z.tif'),
 PosixPath('output/dades_finals_sense_filtrar/openEO_2023-01-14Z

#### ↓ Sentinel2_L2A sense nuvols ↓

In [32]:
# Creem una variable filtre (aka: datacube)
datacube = conn.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=spatial_extent,
    temporal_extent= temporal_extent,
    bands=["B02"],
    max_cloud_cover=max_cloud_coverage,
)

datacube = datacube.apply(lambda x: x / 10000) # Normalitzem

result = datacube.save_result("GTiff") # Diferents formats: {'CovJSON', 'CSV', 'JSON', 'PNG', 'netCDF', 'GTiff', 'Parquet'}

# Creating a new job at the back-end by sending the datacube information.
job = result.create_job()
job.start_and_wait()
job.get_results().download_files(directori_temp_sentinel2_sense_nuvols)


0:00:00 Job 'j-2409144dfe744d20b2f56169c7070092': send 'start'
0:00:25 Job 'j-2409144dfe744d20b2f56169c7070092': created (progress 0%)
0:00:30 Job 'j-2409144dfe744d20b2f56169c7070092': created (progress 0%)
0:00:36 Job 'j-2409144dfe744d20b2f56169c7070092': created (progress 0%)
0:00:44 Job 'j-2409144dfe744d20b2f56169c7070092': created (progress 0%)
0:00:54 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:01:07 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:01:22 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:01:42 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:02:08 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:02:38 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:03:15 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:04:02 Job 'j-2409144dfe744d20b2f56169c7070092': running (progress N/A)
0:05:00 Job 'j-2409144dfe744d20b2f56169c7070092': running (progre

KeyboardInterrupt: 

#### ↓ Filtrar les dades ↓

In [24]:
import os
import shutil 

# Fem una llista amb tots els noms dels fitxers del directori sense nuvols
fitxers_sense_nuvols = sorted([f for f in os.listdir(directori_temp_sentinel2_sense_nuvols) if f.endswith('.tif')])
fitxers_dades_finals = sorted([f for f in os.listdir(directori_temp_dades_finals) if f.endswith('.tif')])

# Movem tots els fitxers que estiquin als dos directoris al directori final
for fitxer in fitxers_sense_nuvols:
    if fitxer in fitxers_dades_finals:
        shutil.move(directori_temp_dades_finals+fitxer, directori_dades_finals+fitxer)



openEO_2023-01-02Z.tif
openEO_2023-01-02Z.tif
openEO_2023-01-05Z.tif
openEO_2023-01-05Z.tif
openEO_2023-01-07Z.tif
openEO_2023-01-07Z.tif
openEO_2023-01-10Z.tif
openEO_2023-01-10Z.tif
openEO_2023-01-15Z.tif
openEO_2023-01-15Z.tif
openEO_2023-01-17Z.tif
openEO_2023-01-17Z.tif
openEO_2023-01-20Z.tif
openEO_2023-01-20Z.tif
openEO_2023-01-22Z.tif
openEO_2023-01-22Z.tif
openEO_2023-01-25Z.tif
openEO_2023-01-25Z.tif
openEO_2023-01-27Z.tif
openEO_2023-01-27Z.tif
openEO_2023-01-30Z.tif
openEO_2023-01-30Z.tif
openEO_2023-02-01Z.tif
openEO_2023-02-01Z.tif
openEO_2023-02-04Z.tif
openEO_2023-02-04Z.tif
openEO_2023-02-06Z.tif
openEO_2023-02-06Z.tif
openEO_2023-02-09Z.tif
openEO_2023-02-09Z.tif
openEO_2023-02-11Z.tif
openEO_2023-02-11Z.tif
openEO_2023-02-14Z.tif
openEO_2023-02-14Z.tif
openEO_2023-02-16Z.tif
openEO_2023-02-16Z.tif
openEO_2023-02-19Z.tif
openEO_2023-02-19Z.tif
openEO_2023-02-21Z.tif
openEO_2023-02-21Z.tif
openEO_2023-02-24Z.tif
openEO_2023-02-24Z.tif
openEO_2023-02-26Z.tif
openEO_2023